In [1]:
%load_ext autoreload
%autoreload 2
# imports
from glob import glob
import json
import numpy as np
from plyfile import PlyData, PlyElement
import numpy as np
import pandas as pd
import pyvista as pv
import sys
import os.path as osp
from matplotlib import pyplot as plt
sys.path.append("./segmentation/cutpursuit/pcd-prox-split/grid-graph/python")
from grid_graph import edge_list_to_forward_star # type: ignore
from segmentation.cutpursuit.python.wrappers.cp_d0_dist import cp_d0_dist

# Pre-process the dataset

In [2]:
feat_map = {
    'Red':'red',
    'Green':'green',
    'Blue':'blue',
    'riegl_reflectance':'intensity',
}

# discard
'riegl_amplitude', 
'riegl_deviation', 
'riegl_targetIndex',
'riegl_targetCount', 
'scan_id', 

# labels
label_names = [
    'labels_0', 
    'labels_1', 
    'labels_2',
    'labels_3'
]

In [3]:

raw_path = "/media/hristo/sharedData1/00_Projects/24-10-26_SPP_PCS/02_Datasets/FWF_Subsampled/0.01"
osp.exists(raw_path)

with open(osp.join(raw_path,'class_dict.json'),'r') as f:
    schema = json.load(f)

print(f"DEBUG flattening labels")
all_labels = []
for pcd_fp in glob(osp.join(raw_path,'*','labeled','*.ply')):
    print(osp.basename(pcd_fp))
    pcd = pd.DataFrame(PlyData.read(pcd_fp).elements[0].data)
    labels = pcd[label_names].to_numpy(dtype=np.int64)
    all_labels.append(labels)

    # pos = pcd[['x','y','z']].to_numpy()
    # pos -= pos.mean(axis=0)
    # feats = pcd[feat_map.keys()].rename(columns=feat_map)
    


all_labels = np.concatenate(all_labels,axis=0)

DEBUG flattening labels
2023-08-28_FW_EingangBauing_pointcloud.ply
2024-03-22_FW_Koenigshuegel_pointcloud.ply
2024-04-05_FW_Westbahnhof_01_pointcloud.ply
2024-04-05_FW_Westbahnhof_02_pointcloud.ply
2024-04-05_FW_Westbahnhof_03_pointcloud.ply
2024-04-05_FW_Westbahnhof_04_pointcloud.ply
2024-04-05_FW_Westbahnhof_05_pointcloud.ply
2024-05-10_FW_RWTH_Zentrum_01_pointcloud.ply
2024-07-31_FW_Bruecke_Koenigstr_pointcloud.ply
2024-07-31_FW_Bruecke_Turmstr_pointcloud.ply
2024-08-02_FW_Bruecke_A44_VerlautenheidenerStr_pointcloud.ply
2024-08-02_FW_Bruecke_Deltourserb_pointcloud.ply
2024-08-02_FW_Bruecke_Kasinostrasse_pointcloud.ply
2024-08-02_FW_Bruecke_RotheErde_pointcloud.ply
2024-08-02_FW_Bruecke_Rottstrasse_pointcloud.ply


In [4]:
import yaml
out_dir = "./data/FWF_flat"


# create flat inds
uq, inv = np.unique(all_labels,return_inverse=True,axis=0)

# create flat schema
new_schema = dict()
for i,l in enumerate(uq):
    new_name = []
    for ln, li in zip(label_names, l):
        sch_inv = {v:k for k,v in schema[ln].items()}
        new_name.append(sch_inv[li])
        
    new_name = '-'.join(new_name)
    new_schema[i] = new_name
    

with open(osp.join(out_dir,'schema.yaml'),'w') as f:
    yaml.dump(new_schema, f)


In [5]:
import torch
from typing import Tuple
def build_csr(edge_idx: torch.Tensor, num_nodes: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    sys.path.append("./segmentation/cutpursuit/pcd-prox-split/grid-graph/python")
    from grid_graph import edge_list_to_forward_star
    indptr, indices, reidx = edge_list_to_forward_star(
        num_nodes, edge_idx.T.contiguous().cpu().numpy()
    )
    return indptr.astype(np.uint32), indices.astype(np.uint32), reidx


In [15]:
ignore_names = [
    # "2023-08-28_FW_EingangBauing_pointcloud",
    # "2024-03-22_FW_Koenigshuegel_pointcloud",
    # "2024-04-05_FW_Westbahnhof_01_pointcloud",
    # "2024-04-05_FW_Westbahnhof_02_pointcloud",
    # "2024-04-05_FW_Westbahnhof_03_pointcloud",
    # "2024-04-05_FW_Westbahnhof_04_pointcloud",
    # "2024-04-05_FW_Westbahnhof_05_pointcloud",
    # "2024-05-10_FW_RWTH_Zentrum_01_pointcloud",
    # "2024-07-31_FW_Bruecke_Koenigstr_pointcloud",
    # "2024-07-31_FW_Bruecke_Turmstr_pointcloud",
]

In [16]:


import os
from utils.pointcloud import compute_geometric_feats
from torch_geometric.nn import knn_graph
import pickle
from tqdm import tqdm
import torch

### CONFIG
config_cp_args = dict(
    regularization = 0.8,
    spatial_weight = 100,
    cutoff = 15,
    cp_dif_tol =  1e-2,
    cp_it_max = 20,
    split_damp_ratio = 0.7
)
config_chunk_size = np.array([10,10])
config_device = 'cuda'

###

all_pcds = dict()
start   = 0
end     = 0

out_dir_processed = osp.join(out_dir,f'{config_chunk_size[0]:.1f}X{config_chunk_size[1]:.1f}')
os.makedirs(out_dir_processed, exist_ok=True)
out_dir_processed_ply = osp.join(out_dir_processed,'ply')
out_dir_processed_graph = osp.join(out_dir_processed,'graph')
os.makedirs(out_dir_processed_ply, exist_ok=True)
os.makedirs(out_dir_processed_graph, exist_ok=True)


# second iteration with flat inds
for area_i, pcd_fp in enumerate(glob(osp.join(raw_path,'*','labeled','*.ply'))):
    
    pcd_name = osp.basename(pcd_fp).replace('.ply','')
    if pcd_name in ignore_names:
        continue

    pcd = pd.DataFrame(PlyData.read(pcd_fp).elements[0].data)
    pos = pcd[['x','y','z']].to_numpy()
    pos -= pos.min(axis=0)
    feats = pcd[feat_map.keys()].rename(columns=feat_map)
    
    # slice labels
    end += pos.shape[0]
    labels = inv[start:end]
    assert labels.shape[0] == pos.shape[0]
    start = end

    # extract chunk inds
    c_coord = pos[:,:2] // config_chunk_size
    chunk_uq_coords, chunk_idx = np.unique(c_coord, return_inverse=True, axis=0)

    # slice pcd in chunks
    for chunk_i, chunk_uq_coord in tqdm(enumerate(chunk_uq_coords),desc=f"Area {area_i+1:03}: {osp.basename(pcd_fp)}"):
        pos_c       = pos[chunk_idx == chunk_i].astype(np.float32)
        feats_c     = feats[chunk_idx == chunk_i].to_numpy().astype(np.float32)
        labels_c    = labels[chunk_idx == chunk_i].astype(np.int32)
        
        # center chunk
        pos_c[:,:2] -= config_chunk_size / 2
        pos_c[:,2]  -= pos_c[:,2].min()

        device = config_device if pos_c.shape[0] < 500_000 else 'cpu'
        if pos_c.shape[0] < 3000:
            continue


        pcd_c = pd.DataFrame(np.concat([pos_c, feats_c, labels_c[:,None]], axis=1), columns=['x','y','z'] + list(feat_map.values()) + ['labels'])
        out_name = f"Area_{area_i+1:02}_X{int(chunk_uq_coord[0]):03}-Y{int(chunk_uq_coord[1]):03}"
        PlyData([PlyElement.describe(pcd_c.to_records(index=False),'vertex')]).write(osp.join(out_dir_processed_ply, f"{out_name}.ply"))
        
        # gather and extract graph building features
        edge_index = knn_graph(torch.as_tensor(pos_c).to(device=device), k=10)
        gfeats_c = compute_geometric_feats(torch.as_tensor(pos_c).to(device=device), edge_index, feat_names=[
            'normals', 
            'verticality', 
            'linearity', 
            'planarity', 
            'scattering', 
        ])      
        pfeats_c = np.concatenate([
            feats_c,
            gfeats_c['normals'].cpu().numpy(), 
            gfeats_c['verticality'][:,None].cpu().numpy(), 
            gfeats_c['linearity'][:,None].cpu().numpy(), 
            gfeats_c['planarity'][:,None].cpu().numpy(), 
            gfeats_c['scattering'][:,None].cpu().numpy(), 
            pos_c[:,2][:,None],
            
        ],axis=1)

        # normalize
        pfeats_c -= pfeats_c.mean(axis=0)
        pfeats_c /= (np.std(pfeats_c, axis=0) + 1e-6)

        # build graph
        Df = pos_c.shape[1] + pfeats_c.shape[1]
        Xf = np.asfortranarray(np.concatenate([pos_c - pos_c.mean(0), pfeats_c], axis=1).T, dtype=np.float64)
        src, tgt, _ = build_csr(edge_index, pos_c.shape[0])


        
        ew = np.ones_like(tgt, np.float64) * config_cp_args.get('regularization', 0.1)
        vw = np.ones(pos_c.shape[0], np.float64)
        cw = np.ones(Df, np.float64)
        cw[:pos_c.shape[1]] *= config_cp_args.get('spatial_weight', 10.0)
        sup, _, graph = cp_d0_dist(
            Df, Xf, src, tgt,
            edge_weights=ew, vert_weights=vw, coor_weights=cw,
            min_comp_weight=config_cp_args.get('cutoff',5),
            cp_dif_tol=config_cp_args.get('cp_dif_tol',1e-2),
            cp_it_max=config_cp_args.get('cp_it_max',10), # type:ignore
            split_damp_ratio=config_cp_args.get('split_damp_ratio',0.7),
            verbose=False, max_num_threads=0,
            balance_parallel_split=True,
            compute_List=False, compute_Graph=True, compute_Time=False
        )

        g_out = dict(
            superpoint_idx = sup,
            edges_forwardstar = graph,
            point_feats = pfeats_c
        )
        with open(osp.join(out_dir_processed_graph,f"{out_name}.pkl"),"wb") as f:
            pickle.dump(g_out, f)



Area 001: 2023-08-28_FW_EingangBauing_pointcloud.ply: 49it [01:35,  1.95s/it]
Area 002: 2024-03-22_FW_Koenigshuegel_pointcloud.ply: 15it [00:14,  1.04it/s]
Area 003: 2024-04-05_FW_Westbahnhof_01_pointcloud.ply: 3it [00:19,  6.42s/it]
Area 004: 2024-04-05_FW_Westbahnhof_02_pointcloud.ply: 2it [00:07,  3.76s/it]
Area 005: 2024-04-05_FW_Westbahnhof_03_pointcloud.ply: 6it [00:17,  2.92s/it]
Area 006: 2024-04-05_FW_Westbahnhof_04_pointcloud.ply: 5it [00:24,  4.87s/it]
Area 007: 2024-04-05_FW_Westbahnhof_05_pointcloud.ply: 1it [00:06,  6.98s/it]
Area 008: 2024-05-10_FW_RWTH_Zentrum_01_pointcloud.ply: 15it [00:46,  3.12s/it]
Area 009: 2024-07-31_FW_Bruecke_Koenigstr_pointcloud.ply: 37it [03:08,  5.10s/it]
Area 010: 2024-07-31_FW_Bruecke_Turmstr_pointcloud.ply: 36it [01:43,  2.89s/it]
Area 011: 2024-08-02_FW_Bruecke_A44_VerlautenheidenerStr_pointcloud.ply: 37it [02:19,  3.78s/it]
Area 012: 2024-08-02_FW_Bruecke_Deltourserb_pointcloud.ply: 26it [00:51,  1.98s/it]
Area 013: 2024-08-02_FW_Bruecke

In [13]:
# import colorsys
# import pyvista as pv

# def index_to_color(indices):
#     """
#     Map integer indices to vibrant RGB colors.

#     Parameters
#     ----------
#     indices : int or array-like of ints
#         Input index or indices. Can be a scalar or an iterable of ints.

#     Returns
#     -------
#     colors : ndarray of shape (N, 3)
#         RGB colors in the range [0, 1], where N is the number of indices.
#     """
#     # Ensure array of ints
#     indices = np.atleast_1d(np.array(indices, dtype=int).flatten())
    
#     # Use the golden ratio conjugate to spread hues evenly
#     phi = 0.618033988749895
#     hues = (indices * phi) % 1.0
    
#     # Vibrancy settings
#     s, v = 0.9, 0.95  # saturation and value
    
#     # Convert each hue to RGB
#     colors = np.array([colorsys.hsv_to_rgb(h, s, v) for h in hues], dtype=float)
#     return colors


# plotter = pv.Plotter(notebook=True)
# colors = index_to_color(sup)
# poly = pv.PolyData(pos_c)
# poly['RGB'] = colors
# plotter.add_mesh(poly, scalars='RGB', rgb=True)
# plotter.show()